# Complete Setup Guide - Warehouse Operational Assistant

This notebook provides a **complete, step-by-step setup guide** from cloning the repository to running the full application with backend and frontend.

## Overview

This guide will walk you through:
1. ✅ Prerequisites verification
2. 📦 Repository setup
3. 🔧 Environment configuration
4. 🔑 NVIDIA API key setup
5. 🗄️ Database setup and migrations
6. 🚀 Starting backend and frontend services
7. ✅ Verification and testing

**Estimated Time:** 30-45 minutes

**Requirements:**
- Python 3.9+
- Node.js 20.0.0+ (or minimum 18.17.0+)
- Docker & Docker Compose (for infrastructure services)
- Git
- NVIDIA API key (free account at https://build.nvidia.com/)

---

## Table of Contents

1. [Prerequisites Check](#prerequisites-check)
2. [Repository Setup](#repository-setup)
3. [Environment Setup](#environment-setup)
4. [API Key Configuration (NVIDIA & Brev)](#api-key-configuration-nvidia--brev)
5. [Environment Variables Setup](#environment-variables-setup)
6. [Infrastructure Services](#infrastructure-services)
7. [Database Setup](#database-setup)
8. [Create Default Users](#create-default-users)
9. [Generate Demo Data](#generate-demo-data)
10. [🚀 (Optional) Install RAPIDS GPU Acceleration](#optional-install-rapids-gpu-acceleration)
11. [Start Backend Server](#start-backend-server)
12. [Start Frontend](#start-frontend)
13. [Verification](#verification)
14. [Troubleshooting](#troubleshooting)


## Step 1: Prerequisites Check

Let's verify that all required tools are installed and meet version requirements.


In [ ]:
import sys
import subprocess
import shutil
from pathlib import Path

def check_command(command, min_version=None, version_flag='--version'):
    """Check if a command exists and optionally verify version."""
    if not shutil.which(command):
        return False, None, f"❌ {command} is not installed"
    
    try:
        result = subprocess.run(
            [command, version_flag],
            capture_output=True,
            text=True,
            timeout=5
        )
        version = result.stdout.strip() or result.stderr.strip()
        return True, version, f"✅ {command} found: {version}"
    except Exception as e:
        return False, None, f"⚠️  {command} found but version check failed: {e}"

def check_python_version():
    """Check Python version."""
    version = sys.version_info
    version_str = f"{version.major}.{version.minor}.{version.micro}"
    
    if version.major < 3 or (version.major == 3 and version.minor < 9):
        return False, version_str, f"❌ Python {version_str} is too old. Required: Python 3.9+"
    return True, version_str, f"✅ Python {version_str} meets requirements"

def check_node_version():
    """Check Node.js version."""
    exists, version, message = check_command('node')
    if not exists:
        return exists, None, message
    
    # Extract version number
    try:
        version_str = version.split()[1] if ' ' in version else version.replace('v', '')
        parts = version_str.split('.')
        major = int(parts[0])
        minor = int(parts[1]) if len(parts) > 1 else 0
        patch = int(parts[2]) if len(parts) > 2 else 0
        
        # Check minimum: 18.17.0, Recommended: 20.0.0+
        if major < 18:
            return False, version_str, f"❌ Node.js {version_str} is too old. Required: 18.17.0+ (Recommended: 20.0.0+)"
        elif major == 18 and (minor < 17 or (minor == 17 and patch < 0)):
            return False, version_str, f"❌ Node.js {version_str} is too old. Required: 18.17.0+ (Recommended: 20.0.0+)"
        elif major == 18:
            return True, version_str, f"⚠️  Node.js {version_str} meets minimum (18.17.0+). Recommended: 20.0.0+"
        else:
            return True, version_str, f"✅ Node.js {version_str} meets requirements (Recommended: 20.0.0+)"
    except:
        return True, version, f"✅ Node.js found: {version}"

print("🔍 Checking Prerequisites...\n")
print("=" * 60)

# Check Python
ok, version, msg = check_python_version()
print(msg)

# Check Node.js
ok, version, msg = check_node_version()
print(msg)

# Check npm
ok, version, msg = check_command('npm')
print(msg)

# Check Git
ok, version, msg = check_command('git')
print(msg)

# Check Docker
ok, version, msg = check_command('docker')
print(msg)

# Check Docker Compose
ok, version, msg = check_command('docker-compose')
if not ok:
    ok, version, msg = check_command('docker', version_flag='compose version')
print(msg)

print("\n" + "=" * 60)
print("\n✅ Prerequisites check complete!")
print("\n📝 If any checks failed, please install the missing tools before proceeding.")


## Step 2: Repository Setup

If you haven't cloned the repository yet, follow the instructions below. If you're already in the repository, you can skip this step.


In [ ]:
import os
from pathlib import Path

# Detect project root: navigate from current directory to find project root
# This handles cases where notebook is opened from notebooks/setup/ or project root
def find_project_root():
    """Find the project root directory."""
    current = Path.cwd()
    
    # Check if we're already in project root
    if (current / "src" / "api").exists() and (current / "scripts" / "setup").exists():
        return current
    
    # Check if we're in notebooks/setup/ (go up 2 levels)
    if (current / "complete_setup_guide.ipynb").exists() or current.name == "setup":
        parent = current.parent.parent
        if (parent / "src" / "api").exists() and (parent / "scripts" / "setup").exists():
            return parent
    
    # Check if we're in notebooks/ (go up 1 level)
    if current.name == "notebooks":
        parent = current.parent
        if (parent / "src" / "api").exists() and (parent / "scripts" / "setup").exists():
            return parent
    
    # Try going up from current directory
    for parent in current.parents:
        if (parent / "src" / "api").exists() and (parent / "scripts" / "setup").exists():
            return parent
    
    # Fallback: return current directory
    return current

# Find and change to project root
project_root = find_project_root()
is_in_repo = (project_root / "src" / "api").exists() and (project_root / "scripts" / "setup").exists()

if is_in_repo:
    # Change to project root so all subsequent operations work correctly
    os.chdir(project_root)
    print("✅ You're already in the Warehouse Operational Assistant repository!")
    print(f"   Project root: {project_root}")
    print(f"   Changed working directory to: {Path.cwd()}")
    print("\n📝 You can skip the cloning step and proceed to environment setup.")
else:
    print("📦 Repository Setup Instructions")
    print("=" * 60)
    print("\nTo clone the repository, run the following command in your terminal:")
    print("\n```bash")
    print("git clone https://github.com/NVIDIA-AI-Blueprints/Multi-Agent-Intelligent-Warehouse.git")
    print("cd Multi-Agent-Intelligent-Warehouse")
    print("```")
    print("\n⚠️  After cloning, restart this notebook from the project root directory.")
    print("\nAlternatively, if you want to clone it now, uncomment and run the cell below:")
    
print(f"\n📁 Current directory: {Path.cwd()}")
print(f"📁 Project root: {project_root}")
print(f"📁 Expected structure: {project_root / 'src' / 'api'}")

In [ ]:
# Uncomment the lines below to clone the repository automatically
# WARNING: This will clone to the current directory

# import subprocess
# 
# repo_url = "https://github.com/NVIDIA-AI-Blueprints/Multi-Agent-Intelligent-Warehouse.git"
# repo_name = "Multi-Agent-Intelligent-Warehouse"
# 
# if not Path(repo_name).exists():
#     print(f"📦 Cloning repository from {repo_url}...")
#     subprocess.run(["git", "clone", repo_url], check=True)
#     print(f"✅ Repository cloned to {Path.cwd() / repo_name}")
#     print(f"\n⚠️  Please change directory and restart this notebook:")
#     print(f"   cd {repo_name}")
#     print(f"   jupyter notebook notebooks/setup/complete_setup_guide.ipynb")
# else:
#     print(f"✅ Repository already exists at {Path.cwd() / repo_name}")

print("💡 To clone manually, use the command shown in the previous cell.")


## Step 3: Environment Setup

This step will:
- Create a Python virtual environment
- Install all Python dependencies
- Verify the installation

### ⚠️ Important: Virtual Environment and Jupyter Kernel

**Best Practice:** For the smoothest experience, create the virtual environment **before** starting Jupyter:

```bash
# Option 1: Create venv first, then start Jupyter (RECOMMENDED)
python3 -m venv env
source env/bin/activate  # or env\Scripts\activate on Windows
pip install jupyter ipykernel
python -m ipykernel install --user --name=warehouse-assistant
jupyter notebook notebooks/setup/complete_setup_guide.ipynb
# Then select "warehouse-assistant" as the kernel
```

**Alternative:** You can create the venv inside this notebook (see below), but you'll need to:
1. Create the venv (this cell)
2. Install ipykernel in the new venv
3. Restart the kernel and switch to the new venv kernel
4. Continue with the rest of the setup

**Note:** The next cell will show which Python/kernel you're currently using.


In [ ]:
import subprocess
import sys
from pathlib import Path

def run_command(cmd, check=True, shell=False):
    """Run a shell command and return the result."""
    if isinstance(cmd, str) and not shell:
        cmd = cmd.split()
    
    result = subprocess.run(
        cmd,
        capture_output=True,
        text=True,
        shell=shell,
        check=check
    )
    return result.returncode == 0, result.stdout, result.stderr

# Show current kernel info
print("🔍 Current Jupyter Kernel Information")
print("=" * 60)
print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version}")
print(f"Working directory: {Path.cwd()}")

# Check if we're already in a virtual environment
in_venv = hasattr(sys, 'real_prefix') or (hasattr(sys, 'base_prefix') and sys.base_prefix != sys.prefix)
if in_venv:
    print(f"✅ Already running in a virtual environment: {sys.prefix}")
    if 'env' in str(sys.prefix) or 'venv' in str(sys.prefix):
        print("   This appears to be the project's virtual environment!")
        use_existing = True
    else:
        print("   ⚠️  This is a different virtual environment")
        use_existing = False
else:
    print("⚠️  Not running in a virtual environment (using system Python)")
    use_existing = False

print("\n" + "=" * 60)

# Check if virtual environment exists
env_path = Path("env")
if env_path.exists():
    print("✅ Virtual environment directory 'env' already exists!")
    
    if use_existing:
        print("✅ You're already using the project's virtual environment - perfect!")
        print("   You can skip the venv creation step and proceed.")
        skip_setup = True
    else:
        print("\n💡 Options:")
        print("   1. Switch to the existing venv kernel (recommended)")
        print("   2. Recreate the virtual environment")
        print("   3. Continue with current kernel (not recommended)")
        
        choice = input("\n❓ What would you like to do? (1/2/3): ").strip()
        
        if choice == '1':
            print("\n📝 To switch kernels:")
            print("   1. Go to: Kernel → Change Kernel → warehouse-assistant")
            print("   2. Or install kernel now:")
            if sys.platform == "win32":
                python_path = Path("env") / "Scripts" / "python.exe"
            else:
                python_path = Path("env") / "bin" / "python"
            
            if python_path.exists():
                print(f"   {python_path} -m ipykernel install --user --name=warehouse-assistant")
                install_kernel = input("\n❓ Install kernel now? (y/N): ").strip().lower()
                if install_kernel == 'y':
                    success, _, _ = run_command([str(python_path), "-m", "ipykernel", "install", "--user", "--name=warehouse-assistant"])
                    if success:
                        print("✅ Kernel installed! Please restart kernel and select 'warehouse-assistant'")
                    else:
                        print("❌ Failed to install kernel")
            skip_setup = True
        elif choice == '2':
            import shutil
            print("🗑️  Removing existing virtual environment...")
            shutil.rmtree(env_path)
            print("✅ Removed")
            skip_setup = False
        else:
            print("⚠️  Continuing with current kernel (may cause issues)")
            skip_setup = True
else:
    skip_setup = False

if not skip_setup:
    print("\n🔧 Setting up Python virtual environment...")
    print("=" * 60)
    
    # Create virtual environment
    print("\n1️⃣ Creating virtual environment...")
    success, stdout, stderr = run_command([sys.executable, "-m", "venv", "env"])
    if success:
        print("✅ Virtual environment created")
    else:
        print(f"❌ Failed to create virtual environment: {stderr}")
        raise RuntimeError("Virtual environment creation failed")
    
    # Determine activation script path
    if sys.platform == "win32":
        activate_script = Path("env") / "Scripts" / "activate"
        pip_path = Path("env") / "Scripts" / "pip"
        python_path = Path("env") / "Scripts" / "python"
    else:
        activate_script = Path("env") / "bin" / "activate"
        pip_path = Path("env") / "bin" / "pip"
        python_path = Path("env") / "bin" / "python"
    
    # Upgrade pip
    print("\n2️⃣ Upgrading pip...")
    success, stdout, stderr = run_command([str(pip_path), "install", "--upgrade", "pip", "setuptools", "wheel"])
    if success:
        print("✅ pip upgraded")
    else:
        print(f"⚠️  pip upgrade had issues: {stderr}")
    
    # Install jupyter and ipykernel in the new venv
    print("\n3️⃣ Installing Jupyter and ipykernel in new environment...")
    success, stdout, stderr = run_command([str(pip_path), "install", "jupyter", "ipykernel"])
    if success:
        print("✅ Jupyter and ipykernel installed")
        
        # Register the kernel
        print("\n4️⃣ Registering kernel...")
        success, stdout, stderr = run_command([str(python_path), "-m", "ipykernel", "install", "--user", "--name=warehouse-assistant"])
        if success:
            print("✅ Kernel 'warehouse-assistant' registered!")
            print("\n⚠️  IMPORTANT: Please restart the kernel and select 'warehouse-assistant'")
            print("   Go to: Kernel → Restart Kernel → Change Kernel → warehouse-assistant")
        else:
            print(f"⚠️  Could not register kernel: {stderr}")
            print("   You can do this manually later")
    else:
        print(f"⚠️  Could not install Jupyter: {stderr}")
    
    # Install requirements
    print("\n5️⃣ Installing Python dependencies...")
    print("   This may take a few minutes...")
    success, stdout, stderr = run_command([str(pip_path), "install", "-r", "requirements.txt"])
    if success:
        print("✅ Dependencies installed successfully")
    else:
        print(f"❌ Failed to install dependencies: {stderr}")
        print("\n💡 Try running manually:")
        print(f"   source env/bin/activate  # or env\\Scripts\\activate on Windows")
        print("   pip install -r requirements.txt")
        raise RuntimeError("Dependency installation failed")
    
    print("\n" + "=" * 60)
    print("⚠️  IMPORTANT NEXT STEP:")
    print("   1. Go to: Kernel → Restart Kernel")
    print("   2. Then: Kernel → Change Kernel → warehouse-assistant")
    print("   3. Re-run this cell to verify you're in the correct environment")
    print("   4. Continue with the rest of the notebook")
else:
    print("\n" + "=" * 60)
    print("✅ Environment setup complete!")
    print("\n📝 Next: Configure environment variables and API keys")


## Step 4: API Key Configuration (NVIDIA & Brev)

The Warehouse Operational Assistant uses NVIDIA NIMs (NVIDIA Inference Microservices) for AI capabilities. You have **two deployment options** for NIMs:

### 🚀 NIM Deployment Options

**Option 1: Cloud Endpoints** (Easiest - Default)
- Use NVIDIA's cloud-hosted NIM services
- **No installation required** - just configure API keys
- Quick setup, perfect for development and testing
- Endpoints: `api.brev.dev` or `integrate.api.nvidia.com`

**Option 2: Self-Hosted NIMs** (Recommended for Production)
- **Install NIMs on your own infrastructure** using Docker
- **Create custom endpoints** on your servers
- Benefits:
  - 🔒 **Data Privacy**: Keep sensitive data on-premises
  - 💰 **Cost Control**: Avoid per-request cloud costs
  - ⚙️ **Custom Requirements**: Full control over infrastructure
  - ⚡ **Low Latency**: Reduced network latency

**Self-Hosting Example:**
```bash
# Deploy LLM NIM on your server
docker run --gpus all -p 8000:8000 \
  nvcr.io/nvidia/nim/llama-3.3-nemotron-super-49b:latest

# Then set in .env:
LLM_NIM_URL=http://your-server:8000/v1
```

**📝 Note**: This step configures API keys for cloud endpoints. If you're self-hosting NIMs, you can skip API keys (unless your NIMs require authentication) and just configure the endpoint URLs in Step 5.

---

### ⚠️ Important: Two Types of API Keys (for Cloud Endpoints)

**1. NVIDIA API Key** (starts with `nvapi-`)
- **Format**: `nvapi-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx`
- **Get from**: https://build.nvidia.com/
- **Works with**: Both `api.brev.dev` and `integrate.api.nvidia.com` endpoints
- **Required for**: Embedding service (always requires NVIDIA API key)

**2. Brev API Key** (starts with `brev_api_`)
- **Format**: `brev_api_xxxxxxxxxxxxxxxxxxxxxxxxxxxxx`
- **Get from**: Your Brev account
- **Works with**: `api.brev.dev` endpoint only
- **Optional**: Can use NVIDIA API key instead

### Configuration Options (Cloud Endpoints)

**Option A: Use NVIDIA API Key for Everything** (Recommended)
- Set `NVIDIA_API_KEY` with your NVIDIA API key
- Leave `EMBEDDING_API_KEY` unset (will use `NVIDIA_API_KEY`)
- Works with both endpoints

**Option B: Use Brev API Key for LLM + NVIDIA API Key for Embedding**
- Set `NVIDIA_API_KEY` with your Brev API key
- **MUST** set `EMBEDDING_API_KEY` with your NVIDIA API key (required!)
- Embedding service always requires NVIDIA API key

### Getting Your API Keys (for Cloud Endpoints)

**NVIDIA API Key:**
1. **Visit**: https://build.nvidia.com/
2. **Sign up** or log in to your NVIDIA account
3. **Navigate** to the "API Keys" section
4. **Create** a new API key
5. **Copy** the API key (starts with `nvapi-`)

**Brev API Key (Optional):**
1. **Visit**: Your Brev account dashboard
2. **Navigate** to API Keys section
3. **Create** or copy your Brev API key (starts with `brev_api_`)

### What You'll Get Access To

- **LLM Service** (Llama 3.3 Nemotron Super 49B) - for chat and reasoning
- **Embedding Service** (llama-3_2-nv-embedqa-1b-v2) - for semantic search
- **Document Processing** - OCR and structured data extraction
- **Content Safety** - NeMo Guardrails for content moderation

**💡 For Self-Hosted NIMs**: See `DEPLOYMENT.md` section "NVIDIA NIMs Deployment & Configuration" for detailed self-hosting instructions.

In [ ]:
import getpass
from pathlib import Path
import re

def setup_api_keys():
    """Interactive setup for API keys (NVIDIA and/or Brev)."""
    print("🔑 API Key Configuration")
    print("=" * 60)
    
    # Check if .env.example exists
    env_example = Path(".env.example")
    if not env_example.exists():
        print("❌ .env.example not found!")
        print("   Please ensure you're in the project root directory.")
        return False
    
    # Check if .env already exists
    env_file = Path(".env")
    if env_file.exists():
        print("✅ .env file already exists")
        overwrite = input("\n❓ Update API keys in existing .env? (y/N): ").strip().lower()
        if overwrite != 'y':
            print("📝 Skipping API key setup. Using existing .env file.")
            return True
    else:
        print("📝 Creating .env file from .env.example...")
        import shutil
        shutil.copy(env_example, env_file)
        print("✅ .env file created")
    
    # Ask about deployment option
    print("\n" + "=" * 60)
    print("🚀 NIM Deployment Options:")
    print("=" * 60)
    print("\n1. Cloud Endpoints (Default - Easiest)")
    print("   • Use NVIDIA's cloud-hosted NIM services")
    print("   • No installation required")
    print("   • Requires API keys (configured below)")
    print("\n2. Self-Hosted NIMs (Advanced)")
    print("   • Install NIMs on your own infrastructure")
    print("   • Create custom endpoints")
    print("   • Better for production, data privacy, cost control")
    print("   • See DEPLOYMENT.md for self-hosting instructions")
    print("=" * 60)
    
    deployment_choice = input("\n❓ Using cloud endpoints or self-hosted NIMs? (1=Cloud, 2=Self-hosted, default: 1): ").strip() or "1"
    
    if deployment_choice == "2":
        print("\n✅ Self-hosted NIMs selected")
        print("   • You can skip API key configuration if your NIMs don't require authentication")
        print("   • Configure endpoint URLs in Step 5 (Environment Variables Setup)")
        print("   • Example: LLM_NIM_URL=http://your-server:8000/v1")
        skip_keys = input("\n❓ Skip API key configuration? (y/N): ").strip().lower()
        if skip_keys == 'y':
            print("📝 Skipping API key setup. Configure endpoints in Step 5.")
            return True
    
    # Get API key configuration choice (for cloud endpoints)
    print("\n" + "=" * 60)
    print("📋 API Key Configuration Options (for Cloud Endpoints):")
    print("=" * 60)
    print("\nOption 1: Use NVIDIA API Key for Everything (Recommended)")
    print("  • Set NVIDIA_API_KEY with NVIDIA API key (nvapi-...)")
    print("  • Leave EMBEDDING_API_KEY unset")
    print("  • Works with both endpoints")
    print("\nOption 2: Use Brev API Key for LLM + NVIDIA API Key for Embedding")
    print("  • Set NVIDIA_API_KEY with Brev API key (brev_api_...)")
    print("  • MUST set EMBEDDING_API_KEY with NVIDIA API key (nvapi-...)")
    print("  • Embedding service always requires NVIDIA API key")
    print("=" * 60)
    
    choice = input("\n❓ Which option? (1 or 2, default: 1): ").strip() or "1"
    
    # Get NVIDIA_API_KEY
    print("\n" + "=" * 60)
    if choice == "1":
        print("📋 Getting NVIDIA API Key:")
        print("1. Visit: https://build.nvidia.com/")
        print("2. Sign up or log in")
        print("3. Go to 'API Keys' section")
        print("4. Create a new API key (starts with 'nvapi-')")
        print("5. Copy the API key")
        print("=" * 60)
        api_key = getpass.getpass("\n🔑 Enter your NVIDIA API key (nvapi-...): ").strip()
        embedding_key = None  # Will use NVIDIA_API_KEY
    else:
        print("📋 Getting Brev API Key for LLM:")
        print("1. Visit: Your Brev account dashboard")
        print("2. Navigate to API Keys section")
        print("3. Create or copy your Brev API key (starts with 'brev_api_')")
        print("=" * 60)
        api_key = getpass.getpass("\n🔑 Enter your Brev API key (brev_api_...): ").strip()
        
        print("\n" + "=" * 60)
        print("📋 Getting NVIDIA API Key for Embedding (REQUIRED):")
        print("1. Visit: https://build.nvidia.com/")
        print("2. Sign up or log in")
        print("3. Go to 'API Keys' section")
        print("4. Create a new API key (starts with 'nvapi-')")
        print("5. Copy the API key")
        print("=" * 60)
        print("⚠️  IMPORTANT: Embedding service REQUIRES NVIDIA API key!")
        embedding_key = getpass.getpass("\n🔑 Enter your NVIDIA API key for Embedding (nvapi-...): ").strip()
    
    if not api_key:
        print("❌ No API key provided. Skipping API key setup.")
        print("   You can set it later in the .env file or environment variables.")
        return False
    
    if api_key.lower() in ["your_nvidia_api_key_here", "your-api-key-here", ""]:
        print("❌ Please enter your actual API key, not the placeholder.")
        return False
    
    # Validate key formats
    if choice == "1" and not api_key.startswith("nvapi-"):
        print("⚠️  Warning: NVIDIA API key should start with 'nvapi-'")
        confirm = input("   Continue anyway? (y/N): ").strip().lower()
        if confirm != 'y':
            return False
    elif choice == "2":
        if not api_key.startswith("brev_api_"):
            print("⚠️  Warning: Brev API key should start with 'brev_api_'")
            confirm = input("   Continue anyway? (y/N): ").strip().lower()
            if confirm != 'y':
                return False
        if not embedding_key or not embedding_key.startswith("nvapi-"):
            print("❌ Embedding service REQUIRES NVIDIA API key (must start with 'nvapi-')")
            return False
    
    # Update .env file
    try:
        with open(env_file, 'r') as f:
            content = f.read()
        
        # Replace NVIDIA_API_KEY
        content = re.sub(
            r'^NVIDIA_API_KEY=.*$',
            f'NVIDIA_API_KEY={api_key}',
            content,
            flags=re.MULTILINE
        )
        
        # Update EMBEDDING_API_KEY if provided
        if embedding_key:
            content = re.sub(
                r'^EMBEDDING_API_KEY=.*$',
                f'EMBEDDING_API_KEY={embedding_key}',
                content,
                flags=re.MULTILINE
            )
        else:
            # Remove EMBEDDING_API_KEY line if using Option 1 (will use NVIDIA_API_KEY)
            content = re.sub(r'^EMBEDDING_API_KEY=.*$\n?', '', content, flags=re.MULTILINE)
        
        # Also update RAIL_API_KEY if it's a placeholder
        if 'RAIL_API_KEY=your_nvidia_api_key_here' in content or 'RAIL_API_KEY=' not in content:
            # Use NVIDIA API key for RAIL (always needs NVIDIA key)
            rail_key = embedding_key if embedding_key else api_key if api_key.startswith("nvapi-") else ""
            if rail_key:
                content = re.sub(
                    r'^RAIL_API_KEY=.*$',
                    f'RAIL_API_KEY={rail_key}',
                    content,
                    flags=re.MULTILINE
                )
        
        with open(env_file, 'w') as f:
            f.write(content)
        
        print("\n✅ API keys configured in .env file")
        if choice == "1":
            print("   • NVIDIA_API_KEY: Set (will be used for all services)")
        else:
            print("   • NVIDIA_API_KEY: Set (Brev API key for LLM)")
            print("   • EMBEDDING_API_KEY: Set (NVIDIA API key for Embedding)")
        print("\n💡 The API keys are stored in .env file (not committed to git)")
        return True
        
    except Exception as e:
        print(f"❌ Error updating .env file: {e}")
        return False

# Run the setup
setup_api_keys()


## Step 5: Environment Variables Setup

Now let's verify and configure other important environment variables. The `.env` file should already be created from the previous step.


In [ ]:
from pathlib import Path
import os
import re

def check_env_file():
    """Check and display environment variable configuration."""
    env_file = Path(".env")
    env_example = Path(".env.example")
    
    if not env_file.exists():
        if env_example.exists():
            print("📝 Creating .env file from .env.example...")
            import shutil
            shutil.copy(env_example, env_file)
            print("✅ .env file created")
        else:
            print("❌ Neither .env nor .env.example found!")
            return False
    
    # Load and display key variables
    print("📋 Environment Variables Configuration")
    print("=" * 60)
    
    with open(env_file, 'r') as f:
        content = f.read()
    
    # Extract key variables
    key_vars = {
        'NVIDIA_API_KEY': 'NVIDIA API Key (for NIM services)',
        'LLM_NIM_URL': 'LLM NIM Endpoint',
        'EMBEDDING_NIM_URL': 'Embedding NIM Endpoint',
        'POSTGRES_PASSWORD': 'Database Password',
        'JWT_SECRET_KEY': 'JWT Secret Key (for authentication)',
        'DEFAULT_ADMIN_PASSWORD': 'Default Admin Password',
        'DB_HOST': 'Database Host',
        'DB_PORT': 'Database Port',
    }
    
    print("\n🔍 Current Configuration:\n")
    for var, description in key_vars.items():
        match = re.search(rf'^{var}=(.*)$', content, re.MULTILINE)
        if match:
            value = match.group(1).strip()
            # Mask sensitive values
            if 'PASSWORD' in var or 'SECRET' in var or 'API_KEY' in var:
                if value and value not in ['changeme', 'your_nvidia_api_key_here', '']:
                    display_value = value[:8] + "..." if len(value) > 8 else "***"
                else:
                    display_value = "⚠️  NOT SET (using default/placeholder)"
            else:
                display_value = value if value else "⚠️  NOT SET"
            print(f"  {var:25} = {display_value:30} # {description}")
        else:
            print(f"  {var:25} = ⚠️  NOT FOUND              # {description}")
    
    print("\n" + "=" * 60)
    print("\n✅ Environment file check complete!")
    print("\n💡 Important Notes:")
    print("   - For production, change all default passwords and secrets")
    print("   - NVIDIA_API_KEY is required for AI features")
    print("   - JWT_SECRET_KEY is required in production")
    print("\n📝 To edit: nano .env  (or your preferred editor)")
    
    return True

# Check environment file
check_env_file()


## Step 6: Infrastructure Services

The application requires several infrastructure services:
- **TimescaleDB** (PostgreSQL with time-series extensions) - Database
- **Redis** - Caching layer
- **Milvus** - Vector database for embeddings
- **Kafka** - Message broker

We'll use Docker Compose to start these services.


In [ ]:
import subprocess
import time
from pathlib import Path

def check_docker_running():
    """Check if Docker is running."""
    try:
        result = subprocess.run(
            ["docker", "info"],
            capture_output=True,
            text=True,
            timeout=5
        )
        return result.returncode == 0
    except:
        return False

def start_infrastructure():
    """Start infrastructure services using Docker Compose."""
    print("🐳 Starting Infrastructure Services")
    print("=" * 60)
    
    if not check_docker_running():
        print("❌ Docker is not running!")
        print("   Please start Docker Desktop or Docker daemon and try again.")
        return False
    
    compose_file = Path("deploy/compose/docker-compose.dev.yaml")
    if not compose_file.exists():
        print(f"❌ Docker Compose file not found: {compose_file}")
        return False
    
    print("\n1️⃣ Checking for existing containers...")
    # Check if docker-compose or docker compose is available
    try:
        result = subprocess.run(
            ["docker", "compose", "version"],
            capture_output=True,
            text=True,
            timeout=5
        )
        compose_cmd = ["docker", "compose"]
    except:
        compose_cmd = ["docker-compose"]
    
    print(f"   Using: {' '.join(compose_cmd)}")
    
    print("\n2️⃣ Starting infrastructure services...")
    print("   This may take a few minutes on first run (downloading images)...")
    
    result = subprocess.run(
        compose_cmd + [
            "-f", str(compose_file),
            "up", "-d"
        ],
        capture_output=True,
        text=True
    )
    
    if result.returncode == 0:
        print("✅ Infrastructure services started")
    else:
        print(f"❌ Failed to start services: {result.stderr}")
        return False
    
    print("\n3️⃣ Waiting for services to be ready...")
    print("   (This may take 30-60 seconds)")
    
    # Wait for TimescaleDB
    max_wait = 60
    waited = 0
    while waited < max_wait:
        try:
            result = subprocess.run(
                ["docker", "exec", "wosa-timescaledb", "pg_isready", "-U", "warehouse"],
                capture_output=True,
                timeout=5
            )
            if result.returncode == 0:
                print("✅ TimescaleDB is ready")
                break
        except:
            pass
        time.sleep(2)
        waited += 2
        if waited % 10 == 0:
            print(f"   Waiting... ({waited}s)")
    
    if waited >= max_wait:
        print("⚠️  TimescaleDB may not be ready yet. Continuing anyway...")
    
    print("\n" + "=" * 60)
    print("✅ Infrastructure services are running!")
    print("\n📋 Service Endpoints:")
    print("   • TimescaleDB: localhost:5435")
    print("   • Redis: localhost:6379")
    print("   • Milvus: localhost:19530 (gRPC), localhost:9091 (HTTP)")
    print("   • Kafka: localhost:9092")
    
    return True

# Uncomment to start infrastructure automatically
# start_infrastructure()

print("💡 To start infrastructure services, run:")
print("   ./scripts/setup/dev_up.sh")
print("\n   Or uncomment the start_infrastructure() call above.")


## Step 7: Database Setup

Now we'll run database migrations to set up the schema. This includes:
- Core schema
- Equipment schema
- Document schema
- Inventory movements schema
- Model tracking tables


In [ ]:
import subprocessimport osfrom pathlib import Pathfrom dotenv import load_dotenv# Load environment variablesload_dotenv()def run_migration(sql_file):    """Run a single SQL migration file.        Tries methods in order:    1. docker-compose exec (recommended - no psql client needed)    2. docker exec (fallback)    3. psql from host (requires PostgreSQL client installed)    """    db_host = os.getenv("DB_HOST", "localhost")    db_port = os.getenv("DB_PORT", "5435")    db_user = os.getenv("POSTGRES_USER", "warehouse")    db_password = os.getenv("POSTGRES_PASSWORD", "changeme")    db_name = os.getenv("POSTGRES_DB", "warehouse")        sql_path = Path(sql_file)    if not sql_path.exists():        return False, f"File not found: {sql_file}"        # Method 1: Try docker-compose exec first (recommended)    try:        result = subprocess.run(            [                "docker-compose", "-f", "deploy/compose/docker-compose.dev.yaml",                "exec", "-T", "timescaledb",                "psql", "-U", db_user, "-d", db_name            ],            input=sql_path.read_text(),            capture_output=True,            text=True,            timeout=30        )        if result.returncode == 0:            return True, "Success"    except FileNotFoundError:        pass  # docker-compose not found, try next method    except Exception as e:        pass  # Try next method        # Method 2: Try docker exec (fallback)    try:        result = subprocess.run(            [                "docker", "exec", "-i", "wosa-timescaledb",                "psql", "-U", db_user, "-d", db_name            ],            input=sql_path.read_text(),            capture_output=True,            text=True,            timeout=30        )        if result.returncode == 0:            return True, "Success"    except FileNotFoundError:        pass  # docker not found, try next method    except Exception as e:        pass  # Try next method        # Method 3: Fall back to psql from host (requires PostgreSQL client)    try:        env = os.environ.copy()        env["PGPASSWORD"] = db_password        result = subprocess.run(            [                "psql",                "-h", db_host,                "-p", db_port,                "-U", db_user,                "-d", db_name,                "-f", str(sql_path)            ],            env=env,            capture_output=True,            text=True,            timeout=30        )        if result.returncode == 0:            return True, "Success"        else:            return False, result.stderr    except FileNotFoundError:        return False, "psql not found. Install PostgreSQL client or use Docker Compose method."    except Exception as e:        return False, f"All methods failed: {str(e)}"def setup_database():    """Run all database migrations."""    print("🗄️  Database Setup and Migrations")    print("=" * 60)        migrations = [        ("data/postgres/000_schema.sql", "Core schema"),        ("data/postgres/001_equipment_schema.sql", "Equipment schema"),        ("data/postgres/002_document_schema.sql", "Document schema"),        ("data/postgres/004_inventory_movements_schema.sql", "Inventory movements schema"),        ("scripts/setup/create_model_tracking_tables.sql", "Model tracking tables"),    ]        print("\n📋 Running migrations...\n")        for sql_file, description in migrations:        print(f"  🔄 {description}...", end=" ")        success, message = run_migration(sql_file)        if success:            print("✅")        else:            print(f"❌\n     Error: {message}")            print(f"\n💡 Try running manually:")            print(f"   # Using Docker Compose (recommended):")            print(f"   docker-compose -f deploy/compose/docker-compose.dev.yaml exec -T timescaledb psql -U warehouse -d warehouse < {sql_file}")            print(f"   # Or using psql (requires PostgreSQL client):")            print(f"   PGPASSWORD=${{POSTGRES_PASSWORD:-changeme}} psql -h localhost -p 5435 -U warehouse -d warehouse -f {sql_file}")            return False        print("\n" + "=" * 60)    print("✅ Database migrations completed successfully!")    return True# Run migrationssetup_database()

## Step 8: Create Default Users

Create the default admin user for accessing the application.


In [ ]:
import subprocess
import sys
from pathlib import Path

def create_default_users():
    """Create default admin user."""
    print("👤 Creating Default Users")
    print("=" * 60)
    
    script_path = Path("scripts/setup/create_default_users.py")
    if not script_path.exists():
        print(f"❌ Script not found: {script_path}")
        return False
    
    # Determine Python path
    if sys.platform == "win32":
        python_path = Path("env") / "Scripts" / "python.exe"
    else:
        python_path = Path("env") / "bin" / "python"
    
    if not python_path.exists():
        print(f"❌ Python not found at: {python_path}")
        print("   Make sure virtual environment is set up (Step 3)")
        return False
    
    print("\n🔄 Running user creation script...")
    result = subprocess.run(
        [str(python_path), str(script_path)],
        capture_output=True,
        text=True
    )
    
    if result.returncode == 0:
        print("✅ Default users created successfully")
        print("\n📋 Default Credentials:")
        print("   Username: admin")
        print("   Password: (check DEFAULT_ADMIN_PASSWORD in .env, default: 'changeme')")
        return True
    else:
        print(f"❌ Failed to create users: {result.stderr}")
        print("\n💡 Try running manually:")
        print(f"   source env/bin/activate  # or env\\Scripts\\activate on Windows")
        print(f"   python {script_path}")
        return False

# Create users
create_default_users()


## Step 9: Generate Demo Data (Optional)

Generate sample data for testing and demonstration purposes. This includes:
- Equipment assets
- Inventory items
- Historical demand data (for forecasting)


In [ ]:
import subprocess
import sys
from pathlib import Path

def generate_demo_data():
    """Generate demo data for testing."""
    print("📊 Generating Demo Data")
    print("=" * 60)
    
    # Determine Python path
    if sys.platform == "win32":
        python_path = Path("env") / "Scripts" / "python.exe"
    else:
        python_path = Path("env") / "bin" / "python"
    
    if not python_path.exists():
        print(f"❌ Python not found at: {python_path}")
        return False
    
    scripts = [
        ("scripts/data/quick_demo_data.py", "Quick demo data (equipment, inventory)"),
        ("scripts/data/generate_historical_demand.py", "Historical demand data (for forecasting)"),
    ]
    
    for script_path, description in scripts:
        script = Path(script_path)
        if not script.exists():
            print(f"⚠️  Script not found: {script_path} (skipping)")
            continue
        
        print(f"\n🔄 {description}...")
        result = subprocess.run(
            [str(python_path), str(script)],
            capture_output=True,
            text=True
        )
        
        if result.returncode == 0:
            print(f"✅ {description} generated")
        else:
            print(f"⚠️  {description} had issues: {result.stderr[:200]}")
    
    print("\n" + "=" * 60)
    print("✅ Demo data generation complete!")
    print("\n💡 You can skip this step if you don't need demo data.")
    return True

# Generate demo data
generate_demo_data()


## Step 10: 🚀 (Optional) Install RAPIDS GPU Acceleration

**This step is OPTIONAL** but highly recommended if you have an NVIDIA GPU. RAPIDS enables **10-100x faster forecasting** with GPU acceleration.

### Benefits
- ⚡ **10-100x faster** training and inference
- 🎯 **Automatic GPU detection** - Falls back to CPU if GPU unavailable
- 🔄 **Zero code changes** - Works automatically when installed
- 📊 **Full model support** - Random Forest, Linear Regression, SVR via cuML; XGBoost via CUDA

### Requirements
- **NVIDIA GPU** with CUDA 12.x support
- **CUDA Compute Capability 7.0+** (Volta, Turing, Ampere, Ada, Hopper)
- **16GB+ GPU memory** (recommended)
- **Python 3.9-3.11**

**Note**: If you don't have a GPU or prefer not to install RAPIDS, you can skip this step. The application will work perfectly on CPU with automatic fallback.


In [ ]:
import subprocess
import sys
from pathlib import Path

def check_gpu_availability():
    """Check if NVIDIA GPU is available."""
    try:
        result = subprocess.run(
            ['nvidia-smi'],
            capture_output=True,
            text=True,
            timeout=5
        )
        if result.returncode == 0:
            return True, result.stdout
        return False, None
    except (FileNotFoundError, subprocess.TimeoutExpired):
        return False, None

def check_rapids_installed():
    """Check if RAPIDS is already installed."""
    try:
        import cudf
        import cuml
        return True, f"cuDF {cudf.__version__}, cuML {cuml.__version__}"
    except ImportError:
        return False, None

def install_rapids():
    """Install RAPIDS cuDF and cuML."""
    print("📦 Installing RAPIDS cuDF and cuML...")
    print("   This may take several minutes (packages are ~2GB)...")
    
    try:
        # Install RAPIDS
        result = subprocess.run(
            [
                sys.executable, '-m', 'pip', 'install',
                '--extra-index-url=https://pypi.nvidia.com',
                'cudf-cu12', 'cuml-cu12'
            ],
            capture_output=True,
            text=True,
            timeout=1800  # 30 minutes timeout
        )
        
        if result.returncode == 0:
            return True, "RAPIDS installed successfully"
        else:
            return False, f"Installation failed: {result.stderr}"
    except subprocess.TimeoutExpired:
        return False, "Installation timed out (took longer than 30 minutes)"
    except Exception as e:
        return False, f"Installation error: {str(e)}"

# Check GPU availability
print("🔍 Checking GPU Availability...")
print("=" * 60)

gpu_available, gpu_info = check_gpu_availability()
if gpu_available:
    print("✅ NVIDIA GPU detected!")
    print("\nGPU Information:")
    print(gpu_info.split('\n')[0:5])  # Show first few lines
    print("\n💡 You can install RAPIDS for GPU acceleration!")
else:
    print("⚠️  NVIDIA GPU not detected or nvidia-smi not available")
    print("   RAPIDS installation is optional - the system will use CPU fallback")

# Check if RAPIDS is already installed
print("\n🔍 Checking RAPIDS Installation...")
print("=" * 60)

rapids_installed, rapids_version = check_rapids_installed()
if rapids_installed:
    print(f"✅ RAPIDS is already installed: {rapids_version}")
    print("   GPU acceleration will be enabled automatically!")
else:
    print("❌ RAPIDS is not installed")
    print("   The system will use CPU fallback (still works great!)")

print("\n" + "=" * 60)
print("\n📝 Next Steps:")
if not rapids_installed and gpu_available:
    print("   • Run the next cell to install RAPIDS (optional but recommended)")
    print("   • Or skip to start the backend server")
elif not gpu_available:
    print("   • GPU not detected - skipping RAPIDS installation")
    print("   • System will use CPU fallback (works perfectly!)")
    print("   • Proceed to start the backend server")
else:
    print("   • RAPIDS is already installed - proceed to start the backend server")


In [ ]:
# OPTIONAL: Install RAPIDS for GPU acceleration
# Uncomment and run this cell if you want to install RAPIDS

# Check if we should install
gpu_available, _ = check_gpu_availability()
rapids_installed, _ = check_rapids_installed()

if rapids_installed:
    print("✅ RAPIDS is already installed - no need to reinstall!")
elif not gpu_available:
    print("⚠️  GPU not detected. RAPIDS installation is not recommended.")
    print("   The system will work perfectly with CPU fallback.")
    print("   If you're sure you have a GPU, you can still install RAPIDS.")
    print("\n   To install anyway, uncomment the install_rapids() call below.")
else:
    print("🚀 Ready to install RAPIDS!")
    print("   This will install:")
    print("   • cuDF (GPU-accelerated DataFrames)")
    print("   • cuML (GPU-accelerated Machine Learning)")
    print("   • Estimated time: 5-15 minutes")
    print("   • Estimated size: ~2GB")
    print("\n   Uncomment the line below to proceed with installation:")
    print("   install_rapids()")

# Uncomment the line below to install RAPIDS:
# success, message = install_rapids()
# if success:
#     print(f"✅ {message}")
#     print("\n🔍 Verifying installation...")
#     rapids_installed, rapids_version = check_rapids_installed()
#     if rapids_installed:
#         print(f"✅ RAPIDS verified: {rapids_version}")
#         print("   GPU acceleration will be enabled automatically!")
#     else:
#         print("⚠️  Installation completed but verification failed")
# else:
#     print(f"❌ {message}")
#     print("\n💡 Don't worry! The system will work perfectly with CPU fallback.")
#     print("   You can try installing RAPIDS later if needed.")


## Step 11: Start Backend Server

Now we'll start the FastAPI backend server. The server will run on port 8001 by default.


In [ ]:
import subprocess
import sys
import time
from pathlib import Path

def check_port(port):
    """Check if a port is in use."""
    import socket
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    result = sock.connect_ex(('localhost', port))
    sock.close()
    return result == 0

def start_backend():
    """Start the backend server."""
    print("🚀 Starting Backend Server")
    print("=" * 60)
    
    port = 8001
    
    # Check if port is already in use
    if check_port(port):
        print(f"⚠️  Port {port} is already in use!")
        print("   The backend may already be running.")
        print(f"   Check: http://localhost:{port}/health")
        return True
    
    # Determine Python path
    if sys.platform == "win32":
        python_path = Path("env") / "Scripts" / "python.exe"
    else:
        python_path = Path("env") / "bin" / "python"
    
    if not python_path.exists():
        print(f"❌ Python not found at: {python_path}")
        return False
    
    print(f"\n🔄 Starting FastAPI server on port {port}...")
    print("   This will run in the background.")
    print("   To stop: Find the process and kill it, or restart the kernel.")
    print("\n📋 Server Endpoints:")
    print(f"   • API: http://localhost:{port}")
    print(f"   • Docs: http://localhost:{port}/docs")
    print(f"   • Health: http://localhost:{port}/health")
    
    # Start server in background
    import threading
    
    def run_server():
        subprocess.run(
            [
                str(python_path),
                "-m", "uvicorn",
                "src.api.app:app",
                "--reload",
                "--port", str(port),
                "--host", "0.0.0.0"
            ],
            cwd=Path.cwd()
        )
    
    server_thread = threading.Thread(target=run_server, daemon=True)
    server_thread.start()
    
    # Wait a bit and check if server started
    print("\n⏳ Waiting for server to start...")
    for i in range(10):
        time.sleep(1)
        if check_port(port):
            print(f"✅ Backend server is running on port {port}!")
            return True
        print(f"   Waiting... ({i+1}/10)")
    
    print("⚠️  Server may still be starting. Check manually:")
    print(f"   curl http://localhost:{port}/health")
    
    return True

print("💡 To start the backend server, you have two options:")
print("\n1️⃣  Run in this notebook (uncomment below):")
print("   # start_backend()")
print("\n2️⃣  Run in a separate terminal (recommended):")
print("   ./scripts/start_server.sh")
print("\n   Or manually:")
print("   source env/bin/activate")
print("   python -m uvicorn src.api.app:app --reload --port 8001 --host 0.0.0.0")


## Step 12: Start Frontend

The frontend is a React application that runs on port 3001. You'll need to install Node.js dependencies first.


In [ ]:
import subprocess
from pathlib import Path

def setup_frontend():
    """Setup and start the frontend."""
    print("🎨 Frontend Setup and Start")
    print("=" * 60)
    
    frontend_dir = Path("src/ui/web")
    if not frontend_dir.exists():
        print(f"❌ Frontend directory not found: {frontend_dir}")
        return False
    
    # Check if node_modules exists
    node_modules = frontend_dir / "node_modules"
    if not node_modules.exists():
        print("\n📦 Installing Node.js dependencies...")
        print("   This may take a few minutes...")
        
        result = subprocess.run(
            ["npm", "install"],
            cwd=frontend_dir,
            capture_output=True,
            text=True
        )
        
        if result.returncode == 0:
            print("✅ Dependencies installed")
        else:
            print(f"❌ Failed to install dependencies: {result.stderr}")
            return False
    else:
        print("✅ Node.js dependencies already installed")
    
    print("\n" + "=" * 60)
    print("✅ Frontend setup complete!")
    print("\n📋 To start the frontend, run in a separate terminal:")
    print(f"   cd {frontend_dir}")
    print("   npm start")
    print("\n   The frontend will be available at: http://localhost:3001")
    print("   Default login: admin / (check DEFAULT_ADMIN_PASSWORD in .env)")
    
    return True

# Setup frontend
setup_frontend()


## Step 13: Verification

Let's verify that everything is set up correctly and the services are running.


In [ ]:
import requests
import subprocess
import socket
from pathlib import Path

def check_service(host, port, name):
    """Check if a service is running on a port."""
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(2)
    result = sock.connect_ex((host, port))
    sock.close()
    return result == 0

def verify_setup():
    """Verify the complete setup."""
    print("✅ Verification Checklist")
    print("=" * 60)
    
    checks = {
        "Virtual Environment": Path("env").exists(),
        "Environment File": Path(".env").exists(),
        "Backend Port (8001)": check_service("localhost", 8001, "Backend"),
        "Frontend Port (3001)": check_service("localhost", 3001, "Frontend"),
        "TimescaleDB (5435)": check_service("localhost", 5435, "TimescaleDB"),
        "Redis (6379)": check_service("localhost", 6379, "Redis"),
        "Milvus (19530)": check_service("localhost", 19530, "Milvus"),
    }
    
    print("\n🔍 Service Status:\n")
    for service, status in checks.items():
        status_icon = "✅" if status else "❌"
        print(f"  {status_icon} {service:25} {'Running' if status else 'Not Running'}")
    
    # Test backend health endpoint
    print("\n🏥 Backend Health Check:")
    try:
        response = requests.get("http://localhost:8001/health", timeout=5)
        if response.status_code == 200:
            print("  ✅ Backend is healthy")
            health_data = response.json()
            if isinstance(health_data, dict):
                print(f"     Status: {health_data.get('status', 'unknown')}")
        else:
            print(f"  ⚠️  Backend returned status {response.status_code}")
    except requests.exceptions.RequestException as e:
        print(f"  ❌ Backend health check failed: {e}")
    
    # Test API endpoint
    print("\n🔌 API Endpoint Check:")
    try:
        response = requests.get("http://localhost:8001/api/v1/version", timeout=5)
        if response.status_code == 200:
            print("  ✅ API is accessible")
            version_data = response.json()
            if isinstance(version_data, dict):
                print(f"     Version: {version_data.get('version', 'unknown')}")
        else:
            print(f"  ⚠️  API returned status {response.status_code}")
    except requests.exceptions.RequestException as e:
        print(f"  ❌ API check failed: {e}")
    
    print("\n" + "=" * 60)
    
    all_checks = all(checks.values())
    if all_checks:
        print("🎉 All checks passed! Your setup is complete!")
    else:
        print("⚠️  Some checks failed. Please review the status above.")
    
    print("\n📋 Access Points:")
    print("   • Frontend: http://localhost:3001")
    print("   • Backend API: http://localhost:8001")
    print("   • API Docs: http://localhost:8001/docs")
    print("   • Health Check: http://localhost:8001/health")
    
    return all_checks

# Run verification
verify_setup()


## Step 14: Troubleshooting

### Common Issues and Solutions

#### 1. Port Already in Use

If a port is already in use, you can either:
- Stop the existing service
- Change the port in the configuration

**Backend (port 8001):**
```bash
# Find and kill process
lsof -ti:8001 | xargs kill -9
# Or change port: export PORT=8002
```

**Frontend (port 3001):**
```bash
# Find and kill process
lsof -ti:3001 | xargs kill -9
# Or change port: PORT=3002 npm start
```

#### 2. Database Connection Errors

**Check if TimescaleDB is running:**
```bash
docker ps | grep timescaledb
```

**Test connection:**
```bash
PGPASSWORD=${POSTGRES_PASSWORD:-changeme} psql -h localhost -p 5435 -U warehouse -d warehouse -c "SELECT 1;"
```

#### 3. Missing Dependencies

**Python:**
```bash
source env/bin/activate
pip install -r requirements.txt
```

**Node.js:**
```bash
cd src/ui/web
npm install
```

#### 4. NVIDIA API Key Issues

- Verify your API key at https://build.nvidia.com/
- Check that `NVIDIA_API_KEY` is set in `.env`
- Test the API key with a curl command (see DEPLOYMENT.md)

#### 5. Node.js Version Issues

If you see `Cannot find module 'node:path'`:
- Upgrade to Node.js 18.17.0+ (recommended: 20.0.0+)
- Check version: `node --version`
- Use nvm to switch versions: `nvm use 20`

### Getting Help

- **Documentation**: See `DEPLOYMENT.md` for detailed deployment guide
- **Issues**: Check GitHub Issues for known problems
- **Logs**: Check service logs for error messages

### Next Steps

1. ✅ Access the frontend at http://localhost:3001
2. ✅ Log in with admin credentials
3. ✅ Explore the features:
   - Chat Assistant
   - Equipment Management
   - Forecasting
   - Operations
   - Safety
   - Document Extraction

**Congratulations! Your Warehouse Operational Assistant is now set up and running! 🎉**


In [ ]:
# Final Summary
print("📋 Setup Summary")
print("=" * 60)
print("\n✅ Completed Steps:")
print("   1. Prerequisites verified")
print("   2. Repository setup")
print("   3. Environment configured")
print("   4. API keys configured")
print("   5. Infrastructure services started")
print("   6. Database migrations completed")
print("   7. Default users created")
print("   8. Demo data generated (optional)")
print("\n🚀 Next Steps:")
print("   1. Start backend: ./scripts/start_server.sh")
print("   2. Start frontend: cd src/ui/web && npm start")
print("   3. Access: http://localhost:3001")
print("\n📚 Documentation:")
print("   • DEPLOYMENT.md - Detailed deployment guide")
print("   • README.md - Project overview")
print("   • docs/ - Additional documentation")
print("\n🎉 Setup complete! Happy coding!")
